## Python parallel processing on Brown University cluster (Oscar).

Use the documentation to supplement this guide https://docs.ccv.brown.edu/oscar/getting-started.

### Objectives

1. Learn how to request a parallel processor 'job'and set up a Python environment that will use that job.
2. Write a short function to test and confirm that parallel processing is taking place.
3. Use these concepts to modify landsatxplore.py from Week09 and implement the NDVI calculation.

### Step 1:  Log in to Oscar vis ssh. Request resources for your compute node.

The [Oscar documentation](https://docs.ccv.brown.edu/oscar/submitting-jobs/slurm) describes how to request cluster computing resources or `jobs`, which are categorized into several distinct partitions. Oscar uses SLURM to manage and allocate resources, but we won't dedicate much time to understanding how SLURM works.  The Python library for distributed processing of array data (dask) will be the tool we focus on.

The `interact` and `batch` commands allows us to request cluster jobs on Oscar.  Documentation for both is found here.  I think you can run all of your work using the `interact` command, as this will simlify the feedback and comprehension loop as you execute your code.  For interact, you need to request the number of nodes `-n`, the amount of time you want the job to last `-t`, and the RAM or memory `-m`.   

The command below requests 5 processors and 150 GB of RAM for 60 minutes.  I find that the fewer processors you request, the faster your job is allocated to you.  **NOTE**: Please do not request more than 24 processors.

~~~
]$ interact -n 5 -t 00:60:00 -m 150g
~~~

~~~
Cores:    5
Walltime: 0:60:00
Memory:   150g
Queue:    batch
salloc -J interact -N 1-1 -n 5 --time=0:60:00 --mem=150g -p batch srun --pty bash
salloc: Pending job allocation 15699369
salloc: job 15699369 queued and waiting for resources
salloc: job 15699369 has been allocated resources
salloc: Granted job allocation 15699369
srun: Step created for job 15699369
module: unloading 'java/8u111'
module: loading 'java/8u111'
module: unloading 'matlab/R2017b'
module: loading 'matlab/R2017b'
module: unloading 'python/3.7.4'
module: loading 'python/2.7.12'
module: unloading 'intel/2017.0'
module: loading 'intel/2017.0'
~~~

**Note** that the modules we loaded (python 3.7 and anacond 3.5) get unloaded when the job is allocated. This is because, Oscar has assigned us a new compute `node` with the processing resources on the computer. Because we are occupying a different physical space in the cluster, our computing environment has been reset to the default.  Before we can do our work, we must reload those as below.

~~~
$ module load python/3.7.4
$ module load anaconda/2020.02
$ source /gpfs/runtime/opt/anaconda/2020.02/etc/profile.d/conda.sh
$ source activate <your_env_here>
~~~

**Aside**.  You can include all of these commands into a text file called a shell script and just run the script to speed up the process.  The script must begin with the line `#!/usr/bin/bash`.  You can use nano to create this shell script.  The protocol is to give it the file extension `.sh`, ie `start.sh`.  After you have created the shell script you need to make it executable with `chmod` command.

~~~
$ chmod a+x start.sh
~~~

The script can be run at the command line using

~~~
$ source start.sh
~~~

### Step 2: Make a coreclock script to confirm parallel processing of computations.

1. Download the script coreclock.py and examine the comments and contents.
1. Add a module called coreclock() to the script, following the comments in the script.
1. Upload the script to Oscar using sftp.
1. Log in to oscar via ssh.
1. Request compute resources for your job following step 2.
1. Load modules, activate your conda environment.

#### Notes about the code in coreclock.py

~~~
	# Client() and LocalCluster() will be used to connect to the job resources that
	# were requested. 
	from dask.distributed import Client, LocalCluster
	# Progress function reports the computation status to the screen
	from dask.distributed import progress
	# Use time library for sleep
	import time
	# Connect to resources.
	cluster = LocalCluster()
	job = Client(cluster)
	print(job)
~~~

The code block above creates a connection to the `interact` job.  The resources can be viewed with `print(job)` 

1. Use client.map() to execute coreclock() on 500 instances of x.
1. Run coreclock.py at the command line:

~~~
$ python coreclock.py
<Client: 'tcp://127.0.0.1:37750' processes=5 threads=5, memory=161.06 GB>
[####                                    ] | 10% Completed | 11.6s
~~~

1. How long does it take for the code to complete execution?
1. Based on 500 instances of coreclock() and the 1 second delay, how long would it take for a single processor to complete the same task?
1. Does processing time scale proportionately with the number of cores? Consider rerunning coreclock with a different number of processors in your interact request, to see if the trend holds.
1. Is parellel computation working as expected?


### Step 3: What to turn in?

* Answer the questions from Step 3 in this .ipynb.
* Upload this .ipynb
* Upload your modified version of coreclock.py 